## Python Code Executor

In this challenge, we have to read the code line by line and then execute them by passing some obstacles.

In [1]:
import re
import io
from typing import Union, TextIO

In [2]:
def _to_pure_code(line: str) -> list[str]:
    """Removes comments and then strips from the right side."""
    return list(filter(lambda x: x, re.split(r'[\s\t]*#.*', line)))

In [7]:
def executor(code: Union[str, TextIO]) -> list[str]:
    """It distinguishes between single-line and multi-line commands."""
    if isinstance(code, io.TextIOBase):
        code = code.read()
    code = re.split(r'\n+', code)

    # If we run a code block line by line, we will encounter an error;
    # so we have to run them after closing.
    codeblock = []
    reports = []

    for idx, line in enumerate(code):
        line = _to_pure_code(line)
        if not line:
            reports.append(f'L#{idx+1:0>2}: Nothing found here')
            continue

        expression = re.match(r'[^\s\t].+[^:]$', line[0])
        if expression is not None:
            if not codeblock:
                reports.append(f'L#{idx+1:0>2}: Just a expression')
                exec(expression.group())
            else:
                reports.append(f'L#{idx+1:0>2}: The code block will be executed first')
                exec('\n'.join(codeblock))
                exec(expression.group())
                codeblock.clear()
            continue

        statement = re.match(r'.+:$', line[0])
        if statement is not None:
            reports.append(f'L#{idx+1:0>2}: A statement found')
            codeblock.append(statement.group())
            continue

        substatement = re.match(r'[\s\t]+.+', line[0])
        if substatement is not None:
            reports.append(f'L#{idx+1:0>2}: This is a sub statement')
            codeblock.append(substatement.group())
            continue

    return reports

In [8]:
code = """
print('Hello, world!')

n = 143  # This is an unnecessary comment
print(f'Ready for {n} repetitions...')
for _ in range(n//25):
    print('I \U00002665 u!')
print(f'Oops! ...The loop is finished!')

print('Goodbye!')
"""
code

"\nprint('Hello, world!')\n\nn = 143  # This is an unnecessary comment\nprint(f'Ready for {n} repetitions...')\nfor _ in range(n//25):\n    print('I ♥ u!')\nprint(f'Oops! ...The loop is finished!')\n\nprint('Goodbye!')\n"

In [9]:
reports = executor(code)

Hello, world!
Ready for 143 repetitions...
I ♥ u!
I ♥ u!
I ♥ u!
I ♥ u!
I ♥ u!
Oops! ...The loop is finished!
Goodbye!


In [10]:
for report in reports:
    print(report)

L#01: Nothing found here
L#02: Just a expression
L#03: Just a expression
L#04: Just a expression
L#05: A statement found
L#06: This is a sub statement
L#07: The code block will be executed first
L#08: Just a expression
L#09: Nothing found here


The worst problem of this implementation is that you can't use `#` and `\n` in a string...